# Demo 2: Scaling relations and volcano plots
In this demonstration you will learn how to plot scaling relations and volcano plots in python. We will use the O and OH adsorption energies from our 2022 transition metal oxides dataset https://www.catalysis-hub.org/publications/ComerUnraveling2022, publication link: https://pubs.acs.org/doi/10.1021/acs.jpcc.2c02381

Start by connecting to the remote server and fetching the pandas dataframe (as covered in previous first demonstration):


In [ ]:
from cathub.cathubsql import CathubSQL
db = CathubSQL()
dataframe = db.get_dataframe(pub_id='ComerUnraveling2022')
dataframe.to_pickle('ComerUnraveling2022.pickle')

Now you can read the data from a local pickle file:

In [ ]:
import pandas
dataframe = pandas.read_pickle('ComerUnraveling2022.pickle')

Start by examining the unique chemical reactions and facets for the dataset, for example:

In [ ]:
print(dataframe[['reaction_id', 'surface_composition', 'facet', 'equation', 'reaction_energy']].to_markdown())

## Atomic structures

Visualize the structures for the 110 and 100 facets. This time, use the ``get_atoms_for_reaction()`` function. This used the ``reaction_id`` entry in the dataframe:

In [ ]:
from ase.visualize import view
atoms_110_Fe = db.get_atoms_for_reaction(436020)
atoms_100_Fe = db.get_atoms_for_reaction(436022)
view(atoms_110_Fe + atoms_100_Fe)

## Scaling relations
Use your favorite python plotting module to plot the OH vs. O scaling (i.e. plotting OH vs O adsorption energies).

tip: Using matplotlib/pylab you can plot the scaling relation like this:

In [ ]:
## import pylab as p

#facet = '100'
loc_O = (dataframe["equation"] =='H2O(g) - H2(g) + * -> O*') 
loc_OH = (dataframe["equation"] =='H2O(g) - 0.5H2(g) + * -> HO*') 

ads_O = dataframe[loc_O]
ads_OH = dataframe[loc_OH]

# Combine O and OH in one dataframe, matching on surface compositions
dataframe_together = ads_O.merge(ads_OH, 
                                 on=['surface_composition', 'facet'],
                                 suffixes=('_O', '_OH'))

x_data = dataframe_together['reaction_energy_OH']
y_data = dataframe_together['reaction_energy_O']
p.figure(figsize=(12,8))
p.scatter(x_data, y_data)
for i, txt in enumerate(dataframe_together['surface_composition']):
    p.gca().annotate(txt,
        (x_data[i],
         y_data[i]))
p.title('O-OH Scaling relation', fontsize=20)
p.xlabel('$\Delta$E$_{OH}$ (eV)', fontsize=16)
p.ylabel('$\Delta$E$_{O}$ (eV)', fontsize=16)

import numpy as np
idx = np.where((-1 < x_data) & (x_data < 1.5))[0]
print(idx)
fit = np.polyfit(x_data[idx],
                 y_data[idx],
                 1)

a = fit[0]
b = fit[1]
xfit = np.array([-2, 3])
yfit = a * xfit + b
             
p.plot(xfit, yfit, '--', label='y={:.2f}x + {:.2f}'.format(a,b))
p.legend(prop={'size':18})

p.show()

## Volcano plots

The Oxygen evolution reaction (OER) overpotential tends to form a volcano relationship with the O-OH adsorption energy difference. 

First we will use scaling relations between OH and OOH to estimate the OOH adsorption energy.


In [ ]:
import pylab as plt
import numpy as np

x_data = dataframe_together['reaction_energy_OH']
y_data = dataframe_together['reaction_energy_O']

def get_overpotential(dGO, dGOH, dGOOH):
    """
    ideal step = 1.23 eV
    
                          __O2__
                   __OOH_|    step4
               _O_|       step3
          _OH_|       step 2  
    _H2O_|      step 1
    """
    step1 = dGOH
    step2 = dGO - dGOH
    step3 = dGOOH - dGO
    step4 = 4 * 1.23 - dGOOH
    step_energies = [step1, step2, step3, step4]
    overpotential = np.max(step_energies) - 1.23
    return overpotential

def ooh_oh_scaling(doh):
    dooh = 0.8443 * doh + 3.136
    return dooh

i = 0
materials = dataframe_together['surface_composition'].values

p.figure(figsize=(8,8))
for dEOH, dEO in zip (x_data, y_data):
    dGO = dEO + 0.023
    dGOH = dEOH + 0.310
    X = dGO - dGOH
    dGOOH = ooh_oh_scaling(dGOH)
    
    overpotential = get_overpotential(dGO, dGOH, dGOOH)
    
    plt.plot(X, overpotential, 'bo')
    
    plt.annotate(materials[i], (X, overpotential))
    i += 1
    

#plt.show()
fit = np.polyfit(x_data + 0.001,
                 y_data + 0.33,
                 1, full=False)

a = 2 #fit[0] 
b = 1 #fit[1]
print(a,b)
c = 0.8443
d = 3.136

x = np.linspace(0, 2.5, 1000)  # O - OH

line_1 = x / (a-1) - b/(a-1) - 1.23  # H2O -> OH step
line_2 = x - 1.23  # OH -> O step
line_3 = (c-a)/(a-1) * x - (c-a) * b / (a-1) + d - b -  1.23  # O -> OOH step
line_4 = 4 * 1.23 - c / (a-1) * x + c*b/(a-1) - d - 1.23 # OOH -> O2 step

volcano_line = []
for i in range(len(x)):
    volcano_line += [np.max([line_2[i], line_3[i]])]

#plt.plot(x, line_1, '--',  color='g')
#plt.plot(x, line_4, '--',  color='r')
plt.plot(x, volcano_line, '--',  color='grey')
plt.ylim(1.2, 0.15)
plt.xlim(0.8, 2.2)
plt.show()

## Challenge: Inspect the correlation between ICOHP and O and OH adsorption energies

The intrgrated Crystal orbital Hamiltonian population (ICOHP) for the M-O bonds in the bulk have been found to be important descriptors for O and OH adsorption energetics. Test the correlation yourself by downloading COHP data from here: https://github.com/SUNCAT-Center/CatHub/tutorials/SUNCAT_summer_institute_2023/

In [ ]:
import json

bulk_cohp_data = json.load(open('bulk_COHP_data.json', 'r'))
dataframe_filter= dataframe[(dataframe['facet']=='100')]
p.figure(figsize=(8,8))
i = 0
for structure, data in bulk_cohp_data.items():
    comp = structure.split('_')[1]
    df = dataframe_filter[(dataframe_filter['surface_composition']== comp + '-rutile')]
    if len(df) == 0:
        continue
    E_OH = df[df["equation"] =='H2O(g) - 0.5H2(g) + * -> HO*']['reaction_energy'].values
    E_O = df[df["equation"] =='H2O(g) - H2(g) + * -> O*']['reaction_energy'].values
    cohp = data['integral']
    if i == 0:
        p.plot(-cohp, E_O, 'bo', label='O')
        p.plot(-cohp, E_OH, 'ro', label='OH')
    else:
        p.plot(-cohp, E_O, 'bo')
        p.plot(-cohp, E_OH, 'ro')
    p.annotate(comp, (-cohp, E_O))
    p.annotate(comp, (-cohp, E_OH))
    i += 1
p.xlabel('Bulk M-O COHP (eV)')
p.ylabel('$\Delta$E (eV)')
p.legend()
p.show()